In [48]:
import numpy as np

# from os import listdir
# from os.path import isfile, join

# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
from keras.preprocessing import sequence

# import PIL.Image

# import json
from tqdm import tqdm

# from utils import *
# import cPickle as pickle
# from matplotlib import pyplot as plt

# from itertools import compress

# import shutil
# import string

# import collections
# import nltk
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import stopwords

# import re
# from numpy.random import random, permutation, randn, normal 

# import os
import preprocessing as preproc


In [32]:
def get_unique_words(captions):
    unique_words = []
    words = [caption.split() for caption in captions]
   
    for word in words:
        unique_words.extend(word)
        
    unique_words = list(set(unique_words))
    
    return unique_words

def get_index_word_dicts(unique_words):
    word_index = {}
    index_word = {}
    for i,word in enumerate(unique_words):
        word_index[word] = i
        index_word[i] = word
        
    return (word_index,index_word)


In [33]:
save_path = "/home/docker/fastai-courses/deeplearning1/nbs/persistent/coco/"

train_path = save_path + "train/"
test_path = save_path + "test/"

images_vgg_4096_folder = "images_vgg_4096/"
captions_folder = "captions/"
indexed_captions_folder = "indexed-captions/"
batch_folder = "batches/"
indexed_future_words_folder = "indexed-future-words/"
misc_folder = "misc/"


# Load Data

In [34]:
base_path = test_path

In [40]:
NR_TRAIN_INSTANCES = 82000
BATCH_SIZE = 500
MAX_CAPTION_LEN = 15

In [41]:
images_precomputed_vgg_features = preproc.read_serialized_np_arr(base_path+images_vgg_4096_folder+'vgg_features.bc'
                                                                  ,nr_instances = NR_TRAIN_INSTANCES )

print(images_precomputed_vgg_features.shape)

(1000, 4096)


In [42]:
raw_captions = preproc.get_truncated_captions_from_batch(base_path + captions_folder, batch_nr = 0,
                                                           nr_instances = NR_TRAIN_INSTANCES )

print(len(raw_captions))

1000


# Save img vgg features batch

In [43]:
for index in tqdm(range(len(images_precomputed_vgg_features) / BATCH_SIZE)):
    img_vgg_feature_list = []
    
    for elem_in_batch in range(BATCH_SIZE):
        img_vgg_feature = np.expand_dims(images_precomputed_vgg_features[index*BATCH_SIZE + elem_in_batch],axis=0)
        img_vgg_feature_list.append(img_vgg_feature)
        
    img_vgg_batch = np.vstack(img_vgg_feature_list)
    save_array(base_path + batch_folder + images_vgg_4096_folder + 'img_vgg_feature_' + str(format(index, "06"))
               + '_' + '.bc',img_vgg_batch)

100%|██████████| 2/2 [00:00<00:00, 28.72it/s]


# Save raw captions batch

In [44]:
for index in tqdm(range(len(raw_captions)/BATCH_SIZE)):
    caption_list = []
    
    for elem_in_batch in range(BATCH_SIZE):
        caption_list.append(raw_captions[index*BATCH_SIZE + elem_in_batch])
     
    captions_batch = np.vstack(caption_list)
    
    save_array(base_path+batch_folder+captions_folder+'caption_'+str(format(index, "06"))+'_'+'.bc',
               captions_batch)

100%|██████████| 2/2 [00:00<00:00, 294.12it/s]


# Save indexed captions batch

In [49]:
unique_words = get_unique_words(raw_captions)
VOCAB_SIZE = len(unique_words)
(word2index, index2word) = get_index_word_dicts(unique_words)

indexed_captions = []
for index,raw_caption in tqdm(enumerate(raw_captions)):
    indexed_caption = [word2index[caption_word] for caption_word in raw_caption.split()]
    indexed_caption = sequence.pad_sequences([indexed_caption], maxlen=MAX_CAPTION_LEN,padding='post')
    indexed_np_arr = np.asarray(np.squeeze(indexed_caption))
    
    indexed_captions.append(indexed_np_arr)
    

1000it [00:00, 27434.01it/s]


In [50]:
for index in tqdm(range(len(indexed_captions)/BATCH_SIZE)):
    
    indexed_caption_list = []
    
    for elem_in_batch in range(BATCH_SIZE):
        indexed_caption = np.expand_dims(indexed_captions[index*BATCH_SIZE + elem_in_batch],axis=0)
        indexed_caption_list.append(indexed_caption)
        
    indexed_captions_batch = np.vstack(indexed_caption_list)
    
    save_array(base_path + batch_folder + indexed_captions_folder + 'indexed_caption_'
               + str(format(index, "06")) + '_' + '.bc',
               indexed_captions_batch)

100%|██████████| 2/2 [00:00<00:00, 142.38it/s]


# Save future words captions batch

In [51]:
for index in tqdm((range(len(indexed_captions)/BATCH_SIZE))):

    indexed_future_word_list = []
        
    for elem_in_batch in range(BATCH_SIZE):

        caption_indexed = indexed_captions[index*BATCH_SIZE + elem_in_batch]
            
        enhanced_caption_indexed = np.append(caption_indexed,[word2index["END"]]) #hacky
        word_2_next_word = []

        for i in xrange(0,len(caption_indexed)):
            caption_word_index = enhanced_caption_indexed[i]
            future_word_index = enhanced_caption_indexed[i+1]
            future_indexes = np.zeros(VOCAB_SIZE)
            future_indexes[future_word_index] = 1

            word_2_next_word.append(future_indexes)

        words_2_next_word = np.vstack(word_2_next_word)
        words_2_next_word = np.expand_dims(words_2_next_word,axis=0)
        
        indexed_future_word_list.append(words_2_next_word)
    
    indexed_future_words_batch = np.vstack(indexed_future_word_list)
    
    save_array(base_path+batch_folder+indexed_future_words_folder+'indexed_future_word_'+str(format(index, "06"))+'_'+'.bc', 
               indexed_future_words_batch)
    


100%|██████████| 2/2 [00:00<00:00,  5.86it/s]


# Saving other misc data 

In [52]:
preproc.save_obj(unique_words,base_path+batch_folder+misc_folder+"unique_words")

In [53]:
preproc.save_obj(word2index,base_path+batch_folder+misc_folder+"word2index")

In [54]:
preproc.save_obj(index2word,base_path+batch_folder+misc_folder+"index2word")